In [3]:
%load_ext autoreload
%autoreload 2
from mt.scan import Scan


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(OpenCL) NVIDIA GeForce RTX 3060 Laptop GPU (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              516.95
	Device Type:                 GPU
	Compute Units:               30
	Global Memory Size:          6143 MB
	Maximum Object Size:         1535 MB
	Max Clock Frequency:         1425 MHz
	Image Support:               Yes

In [1]:
path = "../../04_uCT/AD67/"
scan = Scan(path,
            discard_ends=True)
scan.load(logging=True)

NameError: name 'Scan' is not defined

In [3]:
# scan.try_segmentation_settings()

In [4]:
# scan.try_segmentation_settings(segment_particles_only=True, subset_size=30)

In [5]:
particle_settings = SegmentationSettings(particle_mask_sigma=0.8,
                                particle_n_erosions=7,
                                particle_enlarge_radius=1)
scan.set_particle_segmentation_settings(particle_settings)
print(scan.particle_segmentation_settings)

general_settings = SegmentationSettings(air_mask_sigma=0.6,
                                air_n_erosions=2,
                                contrast_min_percentile=0,
                                contrast_max_percentile=9,
                                particle_mask_sigma=0.2,
                                particle_n_erosions=2,
                                particle_enlarge_radius=1,
                                smooth_labels_radius=2)
scan.set_segmentation_settings(general_settings)
print(scan.segmentation_settings)

air_mask_simga = 0.6
air_n_erosions = 2
particle_mask_sigma = 0.8
particle_n_erosions = 7
particle_enlarge_radius = 1
smooth_labels_radius = 2
contrast_min_percentile = 0
contrast_max_percentile = 100


In [7]:
scan.segment()
scan.segment_particles()
scan.voronoi_tesselation()

In [13]:
scan.show(particle_mask_only=True, axis="z")

In [11]:
scan.save()